In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

In [2]:
olshop = pd.read_csv('/content/olshop_reviews_lexicon.csv')

In [3]:
df = olshop.rename(columns={'sentimen_lexicon': 'labels'})

In [4]:
label2id = {'negatif': 0, 'netral': 1, 'positif': 2}

In [5]:
df['labels'] = df['labels'].map(label2id)

In [6]:
dataset = Dataset.from_pandas(df[['clean_review', 'labels']])

In [7]:
model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_function(examples):
    return tokenizer(examples['clean_review'], truncation=True, padding='max_length', max_length=128)


In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7474 [00:00<?, ? examples/s]

In [10]:
split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = split['train']
eval_dataset = split['test']

In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [16]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-17-804189787>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: galangsatriamahesa-pd (galangsatriamahesa-pd-upn-veteran-yogyakarta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.432000,0.429271
2,0.406400,0.422104
3,0.411900,0.421466


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


TrainOutput(global_step=2244, training_loss=0.4155519590870794, metrics={'train_runtime': 652.6472, 'train_samples_per_second': 27.483, 'train_steps_per_second': 3.438, 'total_flos': 1179866343446784.0, 'train_loss': 0.4155519590870794, 'epoch': 3.0})

In [18]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.42146626114845276, 'eval_model_preparation_time': 0.0049, 'eval_accuracy': 0.8809364548494983, 'eval_precision': 0.7760490374828022, 'eval_recall': 0.8809364548494983, 'eval_f1': 0.8251730519465073, 'eval_runtime': 10.1382, 'eval_samples_per_second': 147.461, 'eval_steps_per_second': 18.445}


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
model.save_pretrained("indobert-sentiment")
tokenizer.save_pretrained("indobert-sentiment")

('indobert-sentiment/tokenizer_config.json',
 'indobert-sentiment/special_tokens_map.json',
 'indobert-sentiment/vocab.txt',
 'indobert-sentiment/added_tokens.json',
 'indobert-sentiment/tokenizer.json')

In [23]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `water_gauge_od_read` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authent

In [25]:
from huggingface_hub import HfApi, HfFolder, Repository, create_repo, upload_folder

repo_name = "indobert-sentiment-tokopedia-shopee"

upload_folder(
    folder_path="indobert-sentiment",
    repo_id=f"galang006/{repo_name}",
    path_in_repo=".",
    commit_message="Upload fine-tuned IndoBERT sentiment model"
)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/galang006/indobert-sentiment-tokopedia-shopee/commit/62d2d8e120da633f9536630cca27eafd6812070f', commit_message='Upload fine-tuned IndoBERT sentiment model', commit_description='', oid='62d2d8e120da633f9536630cca27eafd6812070f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/galang006/indobert-sentiment-tokopedia-shopee', endpoint='https://huggingface.co', repo_type='model', repo_id='galang006/indobert-sentiment-tokopedia-shopee'), pr_revision=None, pr_num=None)

In [35]:
import torch.nn.functional as F

model_id = "indobert-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

id2label = {0: "negatif", 1: "netral", 2: "positif"}

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = F.softmax(logits, dim=-1)
    pred = torch.argmax(probs, dim=-1).item()
    return id2label[pred], probs.squeeze().tolist()

text = input("Masukkan ulasan: ")
label, probas = predict_sentiment(text)
print(f"Prediksi: {label} | Probabilitas: {probas}")

Masukkan ulasan: busuk


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prediksi: negatif | Probabilitas: [0.8996250033378601, 0.010601299814879894, 0.08977377414703369]
